<a href="https://colab.research.google.com/github/wel51x/DS-Unit-4-Sprint-3-Neural-Networks/blob/master/module4-Hyperparameter-Tuning/My_LS_DS_434_Hyperparameter_Tuning_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Your Mission, should you choose to accept it...

To hyperparameter tune and extract every ounce of accuracy out of this telecom customer churn dataset: <https://drive.google.com/file/d/1dfbAsM9DwA7tYhInyflIpZnYs7VT-0AQ/view> 

## Requirements

- Load the data
- Clean the data if necessary (it will be)
- Create and fit a baseline Keras MLP model to the data.
- Hyperparameter tune (at least) the following parameters:
 - batch_size
 - training epochs
 - optimizer
 - learning rate (if applicable to optimizer)
 - momentum (if applicable to optimizer)
 - activation functions
 - network weight initialization
 - dropout regularization
 - number of neurons in the hidden layer
 
 You must use Grid Search and Cross Validation for your initial pass of the above hyperparameters
 
 Try and get the maximum accuracy possible out of this data! You'll save big telecoms millions! Doesn't that sound great?


In [0]:
import pandas as pd
import numpy
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor,  KerasClassifier
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from keras.optimizers import SGD, Adam, Nadam, RMSprop

# fix random seed for reproducibility
numpy.random.seed(42)

In [0]:
#https://raw.githubusercontent.com/wel51x/Data/master/WA_Fn-UseC_-Telco-Customer-Churn.csv
churn_data = pd.read_csv("https://raw.githubusercontent.com/wel51x/Data/master/WA_Fn-UseC_-Telco-Customer-Churn.csv")
churn_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
customerID          7043 non-null object
gender              7043 non-null object
SeniorCitizen       7043 non-null int64
Partner             7043 non-null object
Dependents          7043 non-null object
tenure              7043 non-null int64
PhoneService        7043 non-null object
MultipleLines       7043 non-null object
InternetService     7043 non-null object
OnlineSecurity      7043 non-null object
OnlineBackup        7043 non-null object
DeviceProtection    7043 non-null object
TechSupport         7043 non-null object
StreamingTV         7043 non-null object
StreamingMovies     7043 non-null object
Contract            7043 non-null object
PaperlessBilling    7043 non-null object
PaymentMethod       7043 non-null object
MonthlyCharges      7043 non-null float64
TotalCharges        7043 non-null object
Churn               7043 non-null object
dtypes: float64(1), int64(2), obj

In [0]:
churn_data.sample(8)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
5407,9786-YWNHU,Female,0,Yes,Yes,63,Yes,No,DSL,Yes,...,Yes,Yes,No,No,Two year,No,Mailed check,63.55,4014.2,No
2336,3211-ILJTT,Male,0,Yes,No,17,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Bank transfer (automatic),70.40,1214.05,Yes
5063,3744-ZRRDZ,Male,0,No,No,21,Yes,Yes,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,82.35,1852.85,Yes
1516,6198-RTPMF,Female,0,Yes,No,17,Yes,Yes,Fiber optic,No,...,No,No,Yes,Yes,Month-to-month,No,Electronic check,92.60,1579.7,No
1869,7010-BRBUU,Male,0,Yes,Yes,72,Yes,Yes,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,Two year,No,Credit card (automatic),24.10,1734.65,No
2201,7951-VRDVK,Female,0,No,No,36,Yes,No,DSL,Yes,...,Yes,Yes,Yes,Yes,One year,No,Bank transfer (automatic),85.85,3003.55,No
6985,6967-PEJLL,Male,0,Yes,Yes,37,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,Yes,Electronic check,40.20,1525.35,No
218,2040-LDIWQ,Male,0,Yes,Yes,65,Yes,Yes,DSL,No,...,Yes,Yes,Yes,Yes,Two year,Yes,Bank transfer (automatic),84.20,5324.5,No


In [0]:
churn_data.isnull().sum()

customerID          0
gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64

In [0]:
churn_data.shape

(7032, 21)

### Clean Input Data


In [0]:
# make Total Charges numeric
churn_data['TotalCharges'] = pd.to_numeric(churn_data['TotalCharges'], errors='coerce')
churn_data = churn_data.dropna(subset=['TotalCharges'])

In [0]:
# additional cleaning
churn_data[churn_data.columns] = churn_data[churn_data.columns].replace(('No internet service'), ('No'))
churn_data[churn_data.columns] = churn_data[churn_data.columns].replace(('No phone service'), ('No'))
churn_data['InternetService'] = churn_data['InternetService'].replace(('Fiber optic'), ('Yes'))
churn_data['InternetService'] = churn_data['InternetService'].replace(('DSL'), ('Yes'))
churn_data[churn_data.columns] = churn_data[churn_data.columns].replace(('Yes', 'No'), (1, 0))

In [0]:
churn_data[churn_data.columns] = churn_data[churn_data.columns].replace(('Male', 'Female'), (1, 0))
del churn_data['customerID']

In [0]:
churn_data.sample(8)

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
2481,1,1,1,0,61,1,1,0,0,0,0,0,0,0,2,0,0,25.00,1501.75,0
6784,0,0,0,0,19,1,1,0,0,0,0,0,0,0,0,0,0,24.70,465.85,0
6125,1,0,1,0,13,1,1,1,0,0,1,0,1,1,0,1,1,102.25,1359.00,1
3052,1,0,1,0,37,1,1,1,1,0,0,0,0,0,0,1,3,55.05,2030.75,0
4099,0,0,0,0,6,0,0,1,1,0,0,0,0,0,0,0,0,29.45,161.45,0
3223,1,0,1,0,3,0,0,1,0,1,0,0,0,0,0,0,1,29.80,94.40,0
3774,1,1,1,1,52,1,1,1,0,1,1,0,1,1,1,1,2,80.85,4079.55,0
3469,0,1,0,0,15,1,0,1,0,0,0,0,1,1,0,1,2,91.50,1400.30,0


In [0]:
# encode Contract, PaymentMethod
churn_data["Contract"] = LabelEncoder().fit_transform(churn_data["Contract"])
churn_data["PaymentMethod"] = LabelEncoder().fit_transform(churn_data["PaymentMethod"])

### Normalize Input Data


In [0]:
# Split into X and y and turn into numpy arays
y = churn_data.Churn.values
X = churn_data.drop("Churn", axis='columns').values

# Scale input data
scaler = StandardScaler()
X = scaler.fit_transform(X)
print(X)

[[-1.00943013 -0.44032709  1.03561683 ...  0.39980518 -1.16169394
  -0.99419409]
 [ 0.99065797 -0.44032709 -0.9656081  ...  1.33663626 -0.26087792
  -0.17373982]
 [ 0.99065797 -0.44032709 -0.9656081  ...  1.33663626 -0.36392329
  -0.95964911]
 ...
 [-1.00943013 -0.44032709  1.03561683 ...  0.39980518 -1.17000405
  -0.85451414]
 [ 0.99065797  2.27103902  1.03561683 ...  1.33663626  0.31916782
  -0.87209546]
 [ 0.99065797 -0.44032709 -0.9656081  ... -1.47385696  1.35793167
   2.01234407]]


### Model Validation using an automatic verification Dataset


In [0]:
# Important Hyperparameters
inputs = X.shape[1]
epochs = 50
batch_size = 10

# Create Model
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(inputs,)))
model.add(Dense(64, activation='relu'))
model.add(Dense(1))
# Compile Model
model.compile(optimizer='adam', loss='mse', metrics=['mse', 'accuracy'])
# Fit Model
model.fit(X, y, validation_split=0.2, epochs=epochs, batch_size=batch_size)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Train on 5625 samples, validate on 1407 samples
Epoch 1/50
5625/5625 [==============================] - 1s 237us/step - loss: 0.1673 - mean_squared_error: 0.1673 - acc: 0.7668 - val_loss: 0.1493 - val_mean_squared_error: 0.1493 - val_acc: 0.7868
Epoch 2/50
5625/5625 [==============================] - 1s 127us/step - loss: 0.1428 - mean_squared_error: 0.1428 - acc: 0.7952 - val_loss: 0.1451 - val_mean_squared_error: 0.1451 - val_acc: 0.7960
Epoch 3/50
5625/5625 [==============================] - 1s 120us/step - loss: 0.1366 - mean_squared_error: 0.1366 - acc: 0.8041 - val_loss: 0.1435 - val_mean_squared_error: 0.1435 - val_acc: 0.7967
Epoch 4/50
5625/5625 [==============================] - 1s 117us/step - loss: 0.1337 - mean_squared_error: 0.1337 - acc: 0.8066 - val_loss: 0.1442 - val_mean_squared_error: 0.1442 - val_acc: 0.7982
Epoch 5/50
5625/5625 [==================

In [0]:
scores = model.evaluate(X, y, steps=50)
i = 0
for item in model.metrics_names:
  print(item + ':')
  print(scores[i])
  i += 1

50/50 [==============================] - 0s 6ms/step
loss:
0.09034765511751175
mean_squared_error:
0.09034765511751175
acc:
0.891922652721405


### Manual K-fold Cross Validation (regression)


In [0]:
# Important Hyperparameters
inputs = X.shape[1] # number of cols
epochs = 50
batch_size = 10

# Random Seed
seed = 42
numpy.random.seed(seed)

# define base model
def baseline_model():
  # create model
  model = Sequential()
  model.add(Dense(64, input_dim=inputs, activation='relu')) 
  model.add(Dense(64, activation='relu')) 
  model.add(Dense(1))
  # Compile model
  model.compile(loss='mean_squared_error', optimizer='adam')
  return model

# evaluate model with standardized dataset
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=baseline_model, epochs=epochs,
                                         batch_size=batch_size, verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=5, random_state=seed)
results = cross_val_score(pipeline, X, y, cv=kfold)
print(f"K-fold Cross-Val Results - Mean: {-results.mean():.2f} StDev: {results.std():.2f} MSE")

K-fold Cross-Val Results - Mean: 0.20 StDev: 0.00 MSE


In [0]:
results

array([-0.20511805, -0.19367162, -0.2034683 , -0.19679176, -0.20065535])

### Tune Batch and Epoch Hyperparameters

In [0]:
X.shape

(7032, 19)

In [0]:
# Function to create model, required for KerasClassifier
def create_model():
	# create model
	model = Sequential()
	model.add(Dense(12, input_dim=19, activation='relu'))
	model.add(Dense(1, activation='sigmoid'))
	# Compile model
	model.compile(loss='binary_crossentropy',
                optimizer='adam', metrics=['accuracy'])
	return model

# create model
model = KerasClassifier(build_fn=create_model, verbose=1)

# define the grid search parameters
# batch_size = [10, 20, 40, 60, 80, 100]
# param_grid = dict(batch_size=batch_size, epochs=epochs)

# define the grid search parameters
param_grid = {'batch_size': [10, 20, 50, 80, 100],
              'epochs': [20, 40, 60]}

# Create Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(X, y, verbose=0)

# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}") 

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


Best: 0.8026166044268863 using {'batch_size': 100, 'epochs': 60}
Means: 0.7956484615107716, Stdev: 0.0003483339713811189 with: {'batch_size': 10, 'epochs': 20}
Means: 0.7982081877697043, Stdev: 0.004528350659577554 with: {'batch_size': 10, 'epochs': 40}
Means: 0.7999146723520091, Stdev: 0.0026298694978172596 with: {'batch_size': 10, 'epochs': 60}
Means: 0.7962172938017037, Stdev: 0.0010641807150045324 with: {'batch_size': 20, 'epochs': 20}
Means: 0.795648464435061, Stdev: 0.002764820834560438 with: {'batch_size': 20, 'epochs': 40}
Means: 0.8013367482690952, Stdev: 0.0023193259092087162 with: {'batch_size': 20, 'epochs': 60}
Means: 0.793657561295811, Stdev: 0.0020996622269581636 with: {'batch_size': 50, 'epochs': 20}
Means: 0.7966439070961988, Stdev: 0.0019184753074200226 with: {'batch_size': 50, 'epochs': 40}
Means: 0.7928043185681342, Stdev: 0.002099662950299902 with: {'batch_size': 50, 'epochs': 60}
Means: 0.7918088738557557, Stdev: 0.0019394472149241004 with: {'batch_size': 80, 'epo

## Stretch Goals:

- Try to implement Random Search Hyperparameter Tuning on this dataset
- Try to implement Bayesian Optimiation tuning on this dataset
- Practice hyperparameter tuning other datasets that we have looked at. How high can you get MNIST? Above 99%?
- Study for the Sprint Challenge
 - Can you implement both perceptron and MLP models from scratch with forward and backpropagation?
 - Can you implement both perceptron and MLP models in keras and tune their hyperparameters with cross validation?

### Tune Optimizer Hyperparameters

In [0]:
# Function to create model, add optimizer, learning rate params
def create_opt_model(optimizer=SGD,
                     lr=0.0001):
	# create model
	model = Sequential()
	model.add(Dense(12, input_dim=19, activation='relu'))
	model.add(Dense(1, activation='sigmoid'))
	optimizer=optimizer(lr=lr)
  # Compile model
	model.compile(loss='binary_crossentropy',
                optimizer='adam', metrics=['accuracy'])
	return model


In [57]:
# use 'batch_size': 100, 'epochs': 60 - gave best results in previous step
model = KerasClassifier(build_fn=create_opt_model, batch_size=100,
                        epochs=60, verbose=0)

# define the grid search parameters
param_grid = {'optimizer': [SGD, Adam, RMSprop, Nadam],
              'lr': [.01, .001, .0001, .00001]} # learning rate

# Create Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(X, y)

# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


Best: 0.8020477783490095 using {'lr': 1e-05, 'optimizer': <class 'keras.optimizers.Nadam'>}
Means: 0.7965017060099744, Stdev: 0.003322899524069855 with: {'lr': 0.01, 'optimizer': <class 'keras.optimizers.SGD'>}
Means: 0.7974971566048363, Stdev: 0.0038211104832975348 with: {'lr': 0.01, 'optimizer': <class 'keras.optimizers.Adam'>}
Means: 0.7993458456892745, Stdev: 0.004742130985185013 with: {'lr': 0.01, 'optimizer': <class 'keras.optimizers.RMSprop'>}
Means: 0.7992036370761837, Stdev: 0.003836957631669961 with: {'lr': 0.01, 'optimizer': <class 'keras.optimizers.Nadam'>}
Means: 0.7969283234747604, Stdev: 0.0026298700492007887 with: {'lr': 0.001, 'optimizer': <class 'keras.optimizers.SGD'>}
Means: 0.7973549466694582, Stdev: 0.0019394442242412233 with: {'lr': 0.001, 'optimizer': <class 'keras.optimizers.Adam'>}
Means: 0.7979237760361009, Stdev: 0.0024712930981225217 with: {'lr': 0.001, 'optimizer': <class 'keras.optimizers.RMSprop'>}
Means: 0.8009101230062045, Stdev: 0.0019184738309218612 